In [1]:
DATE = "2021-06-15"
TASK = "100bp-0.1-error-rate"
DIR = "$(DATE)-$(TASK)"
DIR = mkpath("$(homedir())/$(DIR)")

"/Users/cameronprybol/2021-06-15-100bp-0.1-error-rate"

In [47]:
pkgs = [
"LightGraphs",
"MetaGraphs",
"BioSequences",
"uCSV",
"DataFrames",
"FASTX",
"HTTP",
"CodecZlib",
"DataStructures",
"Revise",
"ProgressMeter",
"BenchmarkTools",
"Random",
"StatsBase",
"SparseArrays",
"Statistics",
"BioAlignments"
]

import Pkg
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $(basename(pkg))"))
end

import Mycelia

In [ ]:
# # 11, 17 is too small
# for k in [13, 17, 23, 31, 41, 53]
#     @show k
#     outgraph = "$(DIR)/simplified-kayvirus.k_$(k).all_ncbi.gfa"
#     # if !isfile(outgraph)
#     accessions = ncbi_staph_phage_metadata[!, "Accession"]
#     fastx_iterator = (Mycelia.get_sequence(db="nuccore", accession=accession) for accession in accessions)
#     KMER_TYPE = BioSequences.BigDNAMer{k}
#     @time kmer_graph = Mycelia.fastx_to_kmer_graph(KMER_TYPE, fastx_iterator)

#     @info "simplifying kmer graph"
#     @info "resolving untigs..."
#     @time untigs = Mycelia.resolve_untigs(kmer_graph)

#     @info "determining untig orientations..."
#     @time oriented_untigs = Mycelia.determine_oriented_untigs(kmer_graph, untigs)

#     simplified_graph = MetaGraphs.MetaDiGraph(length(oriented_untigs))
#     MetaGraphs.set_prop!(simplified_graph, :k, kmer_graph.gprops[:k])
#     @info "initializing graph node metadata"
#     ProgressMeter.@showprogress for (vertex, untig) in enumerate(oriented_untigs)
#         MetaGraphs.set_prop!(simplified_graph, vertex, :sequence, untig.sequence)
#         MetaGraphs.set_prop!(simplified_graph, vertex, :path, untig.path)
#         MetaGraphs.set_prop!(simplified_graph, vertex, :orientations, untig.orientations)
#         MetaGraphs.set_prop!(simplified_graph, vertex, :evidence, untig.evidence)
#     end

#     # determine oriented edges of simplified graph
#     simplified_untigs = Vector{Pair{Pair{Int64,Bool},Pair{Int64,Bool}}}(undef, length(LightGraphs.vertices(simplified_graph)))
#     @info "creating simplified unitgs to help resolve connections"
#     # use a pre-allocated array here to speed up
#     ProgressMeter.@showprogress for vertex in LightGraphs.vertices(simplified_graph)
#         in_kmer = simplified_graph.vprops[vertex][:path][1] => simplified_graph.vprops[vertex][:orientations][1]
#         out_kmer = simplified_graph.vprops[vertex][:path][end] => simplified_graph.vprops[vertex][:orientations][end]
#     #     @show vertex, in_kmer, out_kmer
#         simplified_untigs[vertex] = in_kmer => out_kmer
#     #     push!(simplified_untigs, )
#     end

#     # make a dictionary mapping endcap to oriented_untig index

#     end_mer_map = Dict()
#     ProgressMeter.@showprogress for (i, oriented_untig) in enumerate(oriented_untigs)
#         end_mer_map[first(oriented_untig.path)] = i
#         end_mer_map[last(oriented_untig.path)] = i
#     end

#     ProgressMeter.@showprogress for (untig_index, oriented_untig) in enumerate(oriented_untigs)
#     #     @show untig_index
#         true_in_overlap = oriented_untig.sequence[1:simplified_graph.gprops[:k]-1]

#         non_backtracking_neighbors = LightGraphs.neighbors(kmer_graph, oriented_untig.path[1])
#         if length(oriented_untig.path) > 1
#             non_backtracking_neighbors = setdiff(non_backtracking_neighbors, oriented_untig.path[2])
#         end
#         for non_backtracking_neighbor in non_backtracking_neighbors
#             neighboring_untig_index = end_mer_map[non_backtracking_neighbor]
#             neighboring_untig = oriented_untigs[neighboring_untig_index]

#             neighbor_true_out_overlap = neighboring_untig.sequence[end-simplified_graph.gprops[:k]+2:end]
#             if neighbor_true_out_overlap == true_in_overlap
#                 e = LightGraphs.Edge(neighboring_untig_index, untig_index)
#     #             o = true => true
#                 o = (source_orientation = true, destination_orientation = true)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end

#             neighbor_false_out_overlap = BioSequences.reverse_complement(neighboring_untig.sequence)[end-simplified_graph.gprops[:k]+2:end]
#             if neighbor_false_out_overlap == true_in_overlap        
#                 e = LightGraphs.Edge(neighboring_untig_index, untig_index)
#     #             o = false => true
#                 o = (source_orientation = false, destination_orientation = true)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end
#         end

#         true_out_overlap = oriented_untig.sequence[end-simplified_graph.gprops[:k]+2:end]
#         non_backtracking_neighbors = LightGraphs.neighbors(kmer_graph, oriented_untig.path[end])
#         if length(oriented_untig.path) > 1
#             non_backtracking_neighbors = setdiff(non_backtracking_neighbors, oriented_untig.path[end-1])
#         end
#         for non_backtracking_neighbor in non_backtracking_neighbors
#             neighboring_untig_index = end_mer_map[non_backtracking_neighbor]
#             neighboring_untig = oriented_untigs[neighboring_untig_index]

#             neighbor_true_in_overlap = neighboring_untig.sequence[1:simplified_graph.gprops[:k]-1]
#             if true_out_overlap == neighbor_true_in_overlap
#                 e = LightGraphs.Edge(untig_index, neighboring_untig_index)
#     #             o = true => true
#                 o = (source_orientation = true, destination_orientation = true)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end

#             neighbor_false_in_overlap = BioSequences.reverse_complement(neighboring_untig.sequence)[1:simplified_graph.gprops[:k]-1]
#             if true_out_overlap == neighbor_false_in_overlap
#                 e = LightGraphs.Edge(untig_index, neighboring_untig_index)
#     #             o = true => false
#                 o = (source_orientation = true, destination_orientation = false)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end
#         end

#         false_in_overlap = BioSequences.reverse_complement(oriented_untig.sequence)[1:simplified_graph.gprops[:k]-1]

#         non_backtracking_neighbors = LightGraphs.neighbors(kmer_graph, oriented_untig.path[end])
#         if length(oriented_untig.path) > 1
#             non_backtracking_neighbors = setdiff(non_backtracking_neighbors, oriented_untig.path[end-1])
#         end
#         for non_backtracking_neighbor in non_backtracking_neighbors
#             neighboring_untig_index = end_mer_map[non_backtracking_neighbor]
#             neighboring_untig = oriented_untigs[neighboring_untig_index]

#             neighbor_true_out_overlap = neighboring_untig.sequence[end-simplified_graph.gprops[:k]+2:end]
#             if neighbor_true_out_overlap == false_in_overlap
#                 e = LightGraphs.Edge(neighboring_untig_index, untig_index)
#     #             o = true => false
#                 o = (source_orientation = true, destination_orientation = false)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end

#             neighbor_false_out_overlap = BioSequences.reverse_complement(neighboring_untig.sequence)[end-simplified_graph.gprops[:k]+2:end]
#             if neighbor_false_out_overlap == false_in_overlap        
#                 e = LightGraphs.Edge(neighboring_untig_index, untig_index)
#     #             o = false => false
#                 o = (source_orientation = false, destination_orientation = false)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end
#         end

#         false_out_overlap = BioSequences.reverse_complement(oriented_untig.sequence)[end-simplified_graph.gprops[:k]+2:end]

#         non_backtracking_neighbors = LightGraphs.neighbors(kmer_graph, oriented_untig.path[1])
#         if length(oriented_untig.path) > 1
#             non_backtracking_neighbors = setdiff(non_backtracking_neighbors, oriented_untig.path[2])
#         end

#         for non_backtracking_neighbor in non_backtracking_neighbors
#             neighboring_untig_index = end_mer_map[non_backtracking_neighbor]
#             neighboring_untig = oriented_untigs[neighboring_untig_index]

#             neighbor_true_in_overlap = neighboring_untig.sequence[1:simplified_graph.gprops[:k]-1]
#             if false_out_overlap == neighbor_true_in_overlap
#                 e = LightGraphs.Edge(untig_index, neighboring_untig_index)
#     #             o = false => true
#                 o = (source_orientation = false, destination_orientation = true)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end

#             neighbor_false_in_overlap = BioSequences.reverse_complement(neighboring_untig.sequence)[1:simplified_graph.gprops[:k]-1]
#             if false_out_overlap == neighbor_false_in_overlap
#                 e = LightGraphs.Edge(untig_index, neighboring_untig_index)
#     #             o = false => false
#                 o = (source_orientation = false, destination_orientation = false)
#                 LightGraphs.add_edge!(simplified_graph, e)
#                 Mycelia.set_metadata!(simplified_graph, e, :orientations, o)    
#             end
#         end
#     end

#     # ^ > 1 day down to 34 seconds!
#     Mycelia.graph_to_gfa(simplified_graph, outgraph)
# end

In [3]:
# set a random seed
seed = Random.seed!(0)

MersenneTwister(0)

In [4]:
# randomly generate a dna sequence of 100bp
genome = BioSequences.randdnaseq(seed, 100)

100nt DNA Sequence:
AAGGGTGCGGTCTAGGTGCACTGCTTATGGTCCCCGACA…TCTACCTCTTTGTTGAAACGTATTCTTCTCTTAACACCT

In [5]:
# define error rate
error_rate = 0.1

0.1

In [6]:
# generate 100x coverage fastq file
# put accuracy rate into fastq file
coverage = 100
fastq_file = "$(DIR)/$(DATE)-$(TASK).fastq"
open(fastq_file, "w") do io
    fastq_writer = FASTX.FASTQ.Writer(io)
    for i in 1:coverage
        observed_sequence = Mycelia.observe(genome, error_rate=error_rate)
        q = -10 * log10(error_rate)
        quality_scores = fill(q, length(observed_sequence))
        fastq_record = FASTX.FASTQ.Record("i", observed_sequence, quality_scores)
        write(fastq_writer, fastq_record)
    end
end

In [7]:
# build weighted kmer graph
# at some point where we may want to record weights as integers rather than lists of evidence, but first confirm the algorithm

In [8]:
k = Int(1/error_rate + 1)

11

In [9]:
kmer_type = BioSequences.BigDNAMer{k}

BioSequences.BigDNAMer{11} (alias for BioSequences.BigMer{BioSequences.DNAAlphabet{2}, 11})

In [10]:
kmer_graph = Mycelia.fastx_to_kmer_graph(kmer_type, fastq_file)

┌ Info: assessing kmers
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:1877
┌ Info: creating graph
└ @ Mycelia /Users/cameronprybol/.julia/dev/Mycelia/src/Mycelia.jl:1891


{4765, 10236} directed Int64 metagraph with Float64 weights defined by :weight (default weight 1.0)

In [11]:
# visualize
gfa_file = fastq_file * ".k-$k.gfa"
Mycelia.graph_to_gfa(kmer_graph, gfa_file)

"/Users/cameronprybol/2021-06-15-100bp-0.1-error-rate/2021-06-15-100bp-0.1-error-rate.fastq.k-11.gfa"

In [12]:
run(`/Applications/Bandage.app/Contents/MacOS/Bandage image $(gfa_file) $(gfa_file).svg --depwidth 1 --deppower 1`)
# --nodewidth <float> Average node width (0.5 to 1000, default: 5)
# --depwidth <float>  Depth effect on width (0 to 1, default: 0.5)
# --deppower <float>  Power of depth effect on width (0 to 1, default: 0.5)

Process(`/Applications/Bandage.app/Contents/MacOS/Bandage image /Users/cameronprybol/2021-06-15-100bp-0.1-error-rate/2021-06-15-100bp-0.1-error-rate.fastq.k-11.gfa /Users/cameronprybol/2021-06-15-100bp-0.1-error-rate/2021-06-15-100bp-0.1-error-rate.fastq.k-11.gfa.svg --depwidth 1 --deppower 1`, ProcessExited(0))

In [13]:
x = "<img src=../../../../2021-06-15-100bp-0.1-error-rate/2021-06-15-100bp-0.1-error-rate.fastq.k-11.gfa.svg>"
display("text/html", x)
# display("image/svg+xml", read("$(gfa_file).svg", String))

In [14]:
fastq_io = FASTX.FASTQ.Reader(open(fastq_file))

FASTX.FASTQ.Reader{TranscodingStreams.NoopStream{IOStream}}(BioGenerics.Automa.State{TranscodingStreams.NoopStream{IOStream}}(TranscodingStreams.NoopStream{IOStream}(<mode=idle>), 1, 1, false), nothing)

In [15]:
fastq_record = first(fastq_io)

FASTX.FASTQ.Record:
   identifier: i
  description: <missing>
     sequence: AGATGTTAAGAGAGAGAATACTTTTCTAACAAAGTAGGTAATCAGATAATCACGCACGGTCCTGTCGAGGGACCCATAAGCAGTTCGCCAGACCGCACCCTT
      quality: UInt8[0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a  …  0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a, 0x0a]

In [16]:
# for q in FASTX.quality(fastq_read)
#     q = -10 * log10(error_rate)
#     likelihood_error = 10^(q/(-10))
#     display(likelihood_error)
# end

In [17]:
kmers = [kmer_graph.vprops[v][:kmer] for v in LightGraphs.vertices(kmer_graph)]

4765-element Vector{BioSequences.BigDNAMer{11}}:
 AAAAAAGAGGT
 AAAAAAGAGTC
 AAAAACGAATT
 AAAAAGAGCTG
 AAAAAGAGGTA
 AAAAAGAGTCA
 AAAAATACGTT
 AAAAATCGTCT
 AAAACCGTAGT
 AAAACGAATTC
 AAAACTGTATT
 AAAAGAATACG
 AAAAGAGCTGA
 ⋮
 TTGTTAGAAAA
 TTTAAGAGAAA
 TTTAGTTGAAA
 TTTCAAACAAA
 TTTCAACAAAA
 TTTCAAGCAAA
 TTTCACACAAA
 TTTCACAGAAA
 TTTCAGACAAA
 TTTCTAACAAA
 TTTGCAACAAA
 TTTGTTAAAAA

In [18]:
kmer_counts = [length(kmer_graph.vprops[v][:evidence]) for v in LightGraphs.vertices(kmer_graph)]

4765-element Vector{Int64}:
 1
 1
 1
 1
 3
 1
 1
 1
 1
 1
 1
 2
 1
 ⋮
 1
 1
 1
 1
 2
 1
 1
 1
 1
 1
 2
 1

In [19]:
StatsBase.describe(kmer_counts)

Summary Stats:
Length:         4765
Missing Count:  0
Mean:           1.428961
Minimum:        1.000000
1st Quartile:   1.000000
Median:         1.000000
3rd Quartile:   1.000000
Maximum:        15.000000
Type:           Int64


In [20]:
kmer_likelihoods = kmer_counts ./ sum(kmer_counts)

4765-element Vector{Float64}:
 0.000146864444118079
 0.000146864444118079
 0.000146864444118079
 0.000146864444118079
 0.00044059333235423704
 0.000146864444118079
 0.000146864444118079
 0.000146864444118079
 0.000146864444118079
 0.000146864444118079
 0.000146864444118079
 0.000293728888236158
 0.000146864444118079
 ⋮
 0.000146864444118079
 0.000146864444118079
 0.000146864444118079
 0.000146864444118079
 0.000293728888236158
 0.000146864444118079
 0.000146864444118079
 0.000146864444118079
 0.000146864444118079
 0.000146864444118079
 0.000293728888236158
 0.000146864444118079

In [21]:
StatsBase.describe(kmer_likelihoods)

Summary Stats:
Length:         4765
Missing Count:  0
Mean:           0.000210
Minimum:        0.000147
1st Quartile:   0.000147
Median:         0.000147
3rd Quartile:   0.000147
Maximum:        0.002203
Type:           Float64


In [22]:
# evaluate likelihood of first read

In [23]:
kmer_type = BioSequences.BigDNAMer{kmer_graph.gprops[:k]}

BioSequences.BigDNAMer{11} (alias for BioSequences.BigMer{BioSequences.DNAAlphabet{2}, 11})

In [26]:
K = length(LightGraphs.vertices(kmer_graph))

4765

In [27]:
transition_likelihoods = Dict(
    true => SparseArrays.spzeros(K, K),
    false => SparseArrays.spzeros(K, K)
)

for edge in collect(LightGraphs.edges(kmer_graph))
    weight = length(kmer_graph.eprops[edge][:evidence])
    for o in kmer_graph.eprops[edge][:orientations]
        transition_likelihoods[o.source_orientation][edge.src, edge.dst] = weight
    end
end

for source_orientation in (true, false)
    for src in 1:K
        transition_weights = transition_likelihoods[source_orientation][src, :]
        total_weight = sum(transition_weights)
        dsts, vals = SparseArrays.findnz(transition_weights)
        for (dst, val) in zip(dsts, vals) 
            transition_likelihoods[source_orientation][src, dst] = val / total_weight
        end
        normalized_probability = sum(transition_likelihoods[source_orientation][src, :])
        @assert isapprox(normalized_probability, 0) || isapprox(normalized_probability, 1)
    end
end

In [28]:
# evaluate probability of each state

In [30]:
number_of_kmers_in_read = length(FASTX.sequence(fastq_record))-k+1

92

In [ ]:
# maybe don't pre-calculate the shortest paths?
# shortest_paths = LightGraphs.enumerate_paths(LightGraphs.floyd_warshall_shortest_paths(kmer_graph))

In [ ]:
# src = first(LightGraphs.edges(kmer_graph)).src
# dst = first(LightGraphs.edges(kmer_graph)).dst
# LightGraphs.a_star(kmer_graph, src, dst)

In [31]:
state_likelihoods = zeros(K, number_of_kmers_in_read)

4765×92 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [74]:
arrival_paths = fill(Pair{Int, Bool}[], K, number_of_kmers_in_read)

4765×92 Matrix{Vector{Pair{Int64, Bool}}}:
 []  []  []  []  []  []  []  []  []  …  []  []  []  []  []  []  []  []  []
 []  []  []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 []  []  []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 []  []  []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 []  []  []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 []  []  []  []  []  []  []  []  []  …  []  []  []  []  []  []  []  []  []
 []  []  []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 []  []  []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 []  []  []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 []  []  []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 []  []  []  []  []  []  []  []  []  …  []  []  []  []  []  []  []  []  []
 []  []  []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 []  []  []  []  []  []  []  []  []     []  []  []  []  [

In [33]:
initial_state = first(BioSequences.each(kmer_type, FASTX.sequence(fastq_record)))

Mer iteration result:
Position: 1
Forward: AGATGTTAAGA
Backward: TCTTAACATCT


In [35]:
# note this is a place for potential improvement, use the q value at each base to guide probability rather than median
median_q_value = Statistics.median(Int.(FASTX.quality(fastq_record)[1:k]))

10.0

In [36]:
function q_value_to_error_rate(q_value)
    error_rate = 10^(q_value/(-10))
    return error_rate
end

function error_rate_to_q_value(error_rate)
    q_value = -10 * log10(error_rate)
    return q_value
end

error_rate_to_q_value (generic function with 1 method)

In [38]:
current_error_rate = q_value_to_error_rate(median_q_value)

0.1

In [40]:
canonical_kmer = BioSequences.canonical(initial_state.fw)

DNA 11-mer:
AGATGTTAAGA

In [41]:
observed_orientation = initial_state.fw == initial_canonical_kmer

true

In [75]:
x = nothing
for vertex in collect(LightGraphs.vertices(kmer_graph))
    hidden_kmer = kmer_graph.vprops[vertex][:kmer]

    fw_alignment = 
        BioAlignments.pairalign(
            BioAlignments.LevenshteinDistance(), 
            canonical_kmer, 
            hidden_kmer)
    
    fw_probability = 1.0
    
    for match in 1:BioAlignments.count_matches(BioAlignments.alignment(fw_alignment))
        fw_probability *= 1 - error_rate
    end

    for edit in 1:fw_alignment.value
        fw_probability *= error_rate
    end
    
    bw_alignment = 
        BioAlignments.pairalign(
            BioAlignments.LevenshteinDistance(),
            BioSequences.reverse_complement(canonical_kmer),
            hidden_kmer)
    
    bw_probability = 1.0
    
    for match in 1:BioAlignments.count_matches(BioAlignments.alignment(bw_alignment))
        bw_probability *= 1 - error_rate
    end

    for edit in 1:bw_alignment.value
        bw_probability *= error_rate
    end
    
    if fw_probability >= bw_probability
        state_probability = fw_probability
        state_orientation = true
    else
        state_probability = bw_probability
        state_orientation = false
    end
    state_likelihoods[vertex, 1] = state_probability
    arrival_paths[vertex, 1] = [vertex => state_orientation]
end

In [76]:
state_likelihoods

4765×92 Matrix{Float64}:
 6.561e-8    0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 5.9049e-8   0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 6.561e-8    0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 5.9049e-8   0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 5.9049e-8   0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 5.9049e-8   0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 5.9049e-8   0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 5.9049e-7   0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 6.561e-8    0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 7.29e-9     0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 6.561e-8    0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 5.31441e-7  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 5.9049e-7   0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0


In [77]:
arrival_paths

4765×92 Matrix{Vector{Pair{Int64, Bool}}}:
 [1=>1]     []  []  []  []  []  []  []  …  []  []  []  []  []  []  []  []  []
 [2=>0]     []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 [3=>1]     []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 [4=>0]     []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 [5=>1]     []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 [6=>0]     []  []  []  []  []  []  []  …  []  []  []  []  []  []  []  []  []
 [7=>1]     []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 [8=>0]     []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 [9=>1]     []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 [10=>1]    []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 [11=>1]    []  []  []  []  []  []  []  …  []  []  []  []  []  []  []  []  []
 [12=>1]    []  []  []  []  []  []  []     []  []  []  []  []  []  []  []  []
 [13=>1]    []  []  [

In [ ]:
function viterbi_maximum_likelihood_traversals(stranded_kmer_graph;
                                               error_rate::Float64=1/(stranded_kmer_graph.gprops[:k] + 1),
                                               verbosity::String="dataset")




    for K1 in 1:K
        for K2 in 1:K
            if K1 != K2
                shortest_path = shortest_paths[K1][K2]
                path_likelihood = 1.0
                for ui in 1:length(shortest_path)-1
                    u = shortest_path[ui]
                    v = shortest_path[ui + 1]
                    # likelihood of the transition
                    path_likelihood *= edge_probability(stranded_kmer_graph, Edge(u, v))
                end
                if path_likelihood == 0.0
                    shortest_paths[K1][K2] = Vector{Int}()
                end
            elseif K1 == K2
                # the shortest path from a kmer to itself is an insertion (no edge)
                # so need to manually check for self loops
                if has_edge(stranded_kmer_graph, Edge(K1, K2))
                    if edge_probability(stranded_kmer_graph, Edge(K1, K2)) != 0.0
                        shortest_paths[K1][K2] = [K1, K2]
                    else
                        shortest_paths[K1][K2] = Vector{Int}()
                    end
                # otherwise, check to see if any outneighbors connect back to the kmer
                else
                    connected_outneighbors = filter(outneighbor -> has_path(stranded_kmer_graph, outneighbor, K2), outneighbors(stranded_kmer_graph, K1))
                    if !isempty(connected_outneighbors)
                        outneighbor_cycles = [[K1, shortest_paths[outneighbor][K2]...] for outneighbor in connected_outneighbors]
                        cycle_likelihoods = ones(length(outneighbor_cycles))
                        for (i, cycle) in enumerate(outneighbor_cycles)
                            for ui in 1:length(cycle)-1
                                u = cycle[ui]
                                v = cycle[ui + 1]
                                # likelihood of the transition
                                cycle_likelihoods[i] *= edge_probability(stranded_kmer_graph, Edge(u, v))
                            end
                            # include likelihoods of states
                            for vertex in cycle[2:end-1]
                                cycle_likelihoods[i] *= stranded_kmer_likelihoods[vertex]
                            end
                        end
                        path_likelihood = maximum(cycle_likelihoods)
                        max_likelihood_cycle_indices = findall(cycle_likelihoods .== path_likelihood)
                        shortest_paths[K1][K2] = outneighbor_cycles[first(max_likelihood_cycle_indices)]
                    else
                        shortest_paths[K1][K2] = Vector{Int}()
                    end
                end
            end
            if length(shortest_paths[K1][K2]) == 1
                shortest_paths[K1][K2] = Vector{Int}()
            end
        end
    end

    if verbosity in ["debug"]
        for K1 in 1:K
            for K2 in 1:K
                println("\t$K1\t$K2\t$(shortest_paths[K1][K2])")
            end
        end
    end

    total_bases_observed = 0
    total_edits_accepted = 0

    corrected_observations = FASTA.Record[]
    if verbosity in ["debug", "reads", "dataset"]
        println("finding viterbi maximum likelihood paths for observed sequences...")
    end
    # p = Progress(length(stranded_kmer_graph.gprops[:observed_paths]))
    for (observation_index, observed_path) in enumerate(stranded_kmer_graph.gprops[:observed_paths])
        if verbosity in ["debug", "reads"]
            println("\nevaluating sequence $observation_index of $(length(stranded_kmer_graph.gprops[:observed_paths]))")
        end
        # consider switching to log transform
        kmer_likelihoods = zeros(nv(stranded_kmer_graph), length(observed_path))
        kmer_arrival_paths = Array{Vector{Int}}(undef, nv(stranded_kmer_graph), length(observed_path))
        edit_distances = zeros(Int, nv(stranded_kmer_graph), length(observed_path))
        observed_kmer_index = observed_path[1]
        observed_kmer_sequence = stranded_kmer_graph.gprops[:stranded_kmers][observed_kmer_index]
        for hidden_kmer_index in vertices(stranded_kmer_graph)
            hidden_kmer_sequence = stranded_kmer_graph.gprops[:stranded_kmers][hidden_kmer_index]
            alignment_result = pairalign(LevenshteinDistance(), observed_kmer_sequence, hidden_kmer_sequence)
            number_of_matches = count_matches(alignment(alignment_result))
            number_of_edits = stranded_kmer_graph.gprops[:k] - number_of_matches
            kmer_likelihoods[hidden_kmer_index, 1] = stranded_kmer_likelihoods[hidden_kmer_index]
            for match in 1:number_of_matches
                kmer_likelihoods[hidden_kmer_index, 1] *= accuracy
            end
            for edit in 1:number_of_edits
                kmer_likelihoods[hidden_kmer_index, 1] *= error_rate
            end
            kmer_arrival_paths[hidden_kmer_index, 1] = Vector{Int}()
            edit_distances[hidden_kmer_index, 1] = number_of_edits
        end
        kmer_likelihoods[:, 1] ./= sum(kmer_likelihoods[:, 1])
        # from here on, all probabilities are log transformed
        kmer_likelihoods[:, 1] .= log.(kmer_likelihoods[:, 1])
        if verbosity in ["debug"]
            println("\tconsidering path state 1")
            println("\t\tobserved kmer $observed_kmer_sequence")
            println("\t\tInitial state log likelihoods:")
            for line in split(repr(MIME("text/plain"), kmer_likelihoods[:, 1]), '\n')
                println("\t\t\t$line")
            end
        end
        for observed_path_index in 2:length(observed_path)
            observed_kmer_index = observed_path[observed_path_index]
            observed_base = stranded_kmer_graph.gprops[:stranded_kmers][observed_kmer_index][end]

            if verbosity in ["debug"]
                println("\tconsidering path state $observed_path_index")
                println("\t\tobserved base $observed_base")
            end

            MATCH = 1
            MISMATCH = 2
            DELETION = 3
            INSERTION = 4
            arrival_likelihoods = ones(K, 4)
            arrival_paths = fill(Vector{Int}(), K, 4)

            for K2 in 1:K
                kmer_base = stranded_kmer_graph.gprops[:stranded_kmers][K2][end]
                base_is_match = kmer_base == observed_base

                maximum_likelihood = log(0.0)
                maximum_likelihood_path = Vector{Int}()
                maximum_likelihood_edit_distance = 0

                for K1 in 1:K
                    shortest_path = shortest_paths[K1][K2]
                    if length(shortest_path) >= 2
                        edit_distance = Int(!base_is_match) + length(shortest_path) - 2
                        if edit_distance == 0
                            p = kmer_likelihoods[K1, observed_path_index-1] +
                                log(accuracy) + log(stranded_kmer_likelihoods[K2])
                        else
                            p = kmer_likelihoods[K1, observed_path_index-1] +
                                log(error_rate^edit_distance) + log(stranded_kmer_likelihoods[K2])
                        end
                        edit_distance += edit_distances[K1, observed_path_index-1]
                    else
                        p = log(0.0)
                    end
                    if K1 == K2 # consider insertion
                        # in theory, I don't think we should care if the base
                        # matches or not because it's an inserted & erroneous
                        # base, but in practice it's necessary to balance
                        # insertion probabilities with deletion probabilities
                        insertion_p = kmer_likelihoods[K1, observed_path_index-1] +
                                      log(error_rate^(1 + Int(!base_is_match))) + log(stranded_kmer_likelihoods[K2])
                        if insertion_p > p
                            p = insertion_p
                            edit_distance = edit_distances[K1, observed_path_index-1] + 1
                            shortest_path = [K2]
                        end
                    end
                    if p > maximum_likelihood
                        maximum_likelihood = p
                        maximum_likelihood_path = shortest_path
                        maximum_likelihood_edit_distance = edit_distance
                    end
                end
                kmer_likelihoods[K2, observed_path_index] = maximum_likelihood
                kmer_arrival_paths[K2, observed_path_index] = maximum_likelihood_path
                edit_distances[K2, observed_path_index] = maximum_likelihood_edit_distance
            end

            if verbosity in ["debug"]
                println("\t\tkmer log likelihoods")
                for line in split(repr(MIME("text/plain"), kmer_likelihoods), '\n')
                    println("\t\t\t$line")
                end
                println("\t\tarrival paths")
                for line in split(repr(MIME("text/plain"), kmer_arrival_paths), '\n')
                    println("\t\t\t$line")
                end
            end
        end

        if verbosity in ["debug"]
            println("\n\tInputs for viterbi maximum likelihood traversal evaluation:")
            println("\t\tkmer log likelihoods")
            for line in split(repr(MIME("text/plain"), kmer_likelihoods), '\n')
                println("\t\t\t$line")
            end
            println("\t\tkmer arrival paths")
            for line in split(repr(MIME("text/plain"), kmer_arrival_paths), '\n')
                println("\t\t\t$line")
            end
            println("\t\tedit distances")
            for line in split(repr(MIME("text/plain"), edit_distances), '\n')
                println("\t\t\t$line")
            end
        end

        ## backtrack
        maximum_likelihood_path_value = maximum(kmer_likelihoods[:, end])
        maximum_likelihood_path_indices = findall(kmer_likelihoods[:, end] .== maximum_likelihood_path_value)
        # if multiple paths are tied, randomly choose one
        maximum_likelihood_path_index = rand(maximum_likelihood_path_indices)
        maximum_likelihood_edit_distance = edit_distances[maximum_likelihood_path_index, end]

        if length(kmer_arrival_paths[maximum_likelihood_path_index, end]) > 0
            maximum_likelihood_path = last(kmer_arrival_paths[maximum_likelihood_path_index, end])
            for observed_path_index in length(observed_path):-1:1
                maximum_likelihood_arrival_path = kmer_arrival_paths[maximum_likelihood_path_index, observed_path_index]
                maximum_likelihood_path = vcat(maximum_likelihood_arrival_path[1:end-1], maximum_likelihood_path)
                maximum_likelihood_path_index = first(maximum_likelihood_path)
            end
        else
            maximum_likelihood_path = [maximum_likelihood_path_index]
        end
        observed_sequence = path_to_sequence(stranded_kmer_graph.gprops[:stranded_kmers], observed_path)
        maximum_likelihood_sequence = path_to_sequence(stranded_kmer_graph.gprops[:stranded_kmers], maximum_likelihood_path)
        if verbosity in ["debug", "reads"]
            println("\tobserved sequence                 $observed_sequence")
            println("\tmaximum likelihood sequence       $maximum_likelihood_sequence")
            println("\tmaximum likelihood edit distance  $maximum_likelihood_edit_distance")
        end
        total_bases_observed += length(observed_sequence)
        total_edits_accepted += maximum_likelihood_edit_distance
        id = stranded_kmer_graph.gprops[:observation_ids][observation_index]
        kmer_stamped_id = id * "_" * string(stranded_kmer_graph.gprops[:k])
        push!(corrected_observations, FASTA.Record(kmer_stamped_id, maximum_likelihood_sequence))
        # progress meter
        # next!(p)
    end
    if verbosity in ["debug", "reads", "dataset"]
        println("\nDATASET STATISTICS:")
        println("\tassumed error rate    $(error_rate * 100)%")
        println("\ttotal bases observed  $total_bases_observed")
        println("\ttotal edits accepted  $total_edits_accepted")
        println("\tinferred error rate   $((total_edits_accepted/total_bases_observed) * 100)%")
    end
    return corrected_observations
end

In [ ]:
# backtrack most likely path

In [ ]:
# write sequence to output fastq

In [ ]:
# for k in [11, 13, 17, 23, 31, 41, 53]
#     @show k
#     outgraph = "$(DIR)/simplified-kayvirus.k_$(k).all_ncbi.gfa"
#     if !isfile(outgraph)
#         accessions = ncbi_staph_phage_metadata[!, "Accession"]
#         fastx_iterator = (Mycelia.get_sequence(db="nuccore", accession=accession) for accession in accessions)
#         KMER_TYPE = BioSequences.BigDNAMer{k}
#         @time kmer_graph = Mycelia.fastx_to_kmer_graph(KMER_TYPE, fastx_iterator)
#         @time simplified_graph = Mycelia.simplify_kmer_graph(kmer_graph)
# #         ^ > 1 day down to 34 seconds!
#         Mycelia.graph_to_gfa(simplified_graph, outgraph)
#     end
# end